In [107]:
import os
import pandas as pd
import numpy as np
import re

pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)
pd.set_option('display.max_colwidth', -1)

In [108]:
compare_folder = "/home/laura/ANALYSIS/VARIANT_CALLING/COVID/Compare/2020-12-09_COVID"
pairwise_filename = [x for x in os.listdir(compare_folder) if '.snp.pairwise.tsv' in x][0]
pairwise_file = os.path.join(compare_folder, pairwise_filename)
pwdf = pd.read_csv(pairwise_file, sep="\t" , names=['sample_1', 'sample_2', 'dist'])

In [109]:
pwdf.head()

,sample_1,sample_2,dist
0,20461953,20461953,0
1,20461953,20069944,14
2,20461953,20069551,28
3,20461953,20091271,16
4,20461953,20081614,24


In [110]:
pwdf['sample_1'] = pwdf['sample_1'].astype(str)

In [111]:
pwdf.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 984064 entries, 0 to 984063
Data columns (total 3 columns):
sample_1    984064 non-null object
sample_2    984064 non-null object
dist        984064 non-null int64
dtypes: int64(1), object(2)
memory usage: 22.5+ MB


In [112]:
samples = list(set(pwdf.sample_1.tolist()))

In [113]:
len(samples)

992

In [114]:
samples_type = [type(x) for x in samples]
dict((x,samples_type.count(x)) for x in set(samples_type))

{str: 992}

In [115]:
wgs = pd.read_csv('/home/laura/ANALYSIS/MISC/microreact/WGS_COVID.csv')

In [116]:
wgs.head()

,id,full_date,patient__autocolor,province__autocolor,SEX__autocolor,AGE__autocolor,day,month,year
0,20060161,09/03/2020,174.0,Madrid,H,84,9.0,3.0,2020.0
1,20060178,09/03/2020,175.0,Madrid,H,76,9.0,3.0,2020.0
2,20062236,10/03/2020,176.0,Madrid,M,96,10.0,3.0,2020.0
3,20062563,10/03/2020,177.0,Madrid,H,78,10.0,3.0,2020.0
4,20063795,12/03/2020,178.0,Madrid,H,85,12.0,3.0,2020.0


In [117]:
wgs.shape

(1207, 9)

In [118]:
wgs['id'] = wgs['id'].astype(str)

In [119]:
samples_wgs = wgs.id.tolist()
#samples_wgs = [str(x) for x in samples_wgs]

In [120]:
WGS = set(samples_wgs)
PW = set(samples)

In [121]:
len(PW)

992

In [122]:
missing_wgs = list(PW - WGS)
len(missing_wgs)

51

In [123]:
missing_wgs_type = [type(x) for x in missing_wgs]
dict((x,missing_wgs_type.count(x)) for x in set(missing_wgs_type))

{str: 51}

In [124]:
missing_pw = list(WGS - PW)
len(missing_pw)

169

In [125]:
all_missing = missing_pw + missing_wgs
len(all_missing)

220

In [126]:
wgs.shape

(1207, 9)

In [127]:
df = wgs[~wgs.id.isin(missing_pw)]

In [128]:
df.shape

(1032, 9)

In [129]:
df2 = pd.DataFrame(missing_wgs, columns=['id'])

In [130]:
df2['id'] = df2['id'].astype(str)

In [131]:
df2.head()

,id
0,20461389-BIS
1,20174144
2,20289862
3,20273207
4,20095471-8


In [132]:
df.head()

,id,full_date,patient__autocolor,province__autocolor,SEX__autocolor,AGE__autocolor,day,month,year
0,20060161,09/03/2020,174.0,Madrid,H,84,9.0,3.0,2020.0
1,20060178,09/03/2020,175.0,Madrid,H,76,9.0,3.0,2020.0
2,20062236,10/03/2020,176.0,Madrid,M,96,10.0,3.0,2020.0
3,20062563,10/03/2020,177.0,Madrid,H,78,10.0,3.0,2020.0
4,20063795,12/03/2020,178.0,Madrid,H,85,12.0,3.0,2020.0


In [133]:
final_df = df.append(df2).reset_index(drop=True)
final_df = final_df[ ['id', 'full_date', 'patient__autocolor', 'province__autocolor', 'SEX__autocolor',
       'AGE__autocolor', 'day', 'month', 'year'] ]

In [134]:
metadata = pd.read_csv('/home/laura/ANALYSIS/MISC/microreact/wgs_covid_metadata.csv')
metadata['id'] = metadata['id'].astype(str)

In [135]:
metadata.tail()

,id,DEPARTMENT__autocolor
607,20415606,5400
608,20415652,3300
609,20421279,PROFESIONAL_SANITARIO
610,20421333,3300
611,20423264,3400


In [136]:
metadata.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 612 entries, 0 to 611
Data columns (total 2 columns):
id                       612 non-null object
DEPARTMENT__autocolor    612 non-null object
dtypes: object(2)
memory usage: 9.7+ KB


In [137]:
final_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1083 entries, 0 to 1082
Data columns (total 9 columns):
id                     1083 non-null object
full_date              1028 non-null object
patient__autocolor     789 non-null float64
province__autocolor    869 non-null object
SEX__autocolor         1024 non-null object
AGE__autocolor         1025 non-null object
day                    1028 non-null float64
month                  1028 non-null float64
year                   1028 non-null float64
dtypes: float64(4), object(5)
memory usage: 76.3+ KB


In [138]:
final_df = final_df.fillna(0)
final_df.day = final_df.day.astype('Int64')
final_df.month = final_df.month.astype('Int64')
final_df.year = final_df.year.astype('Int64')
final_df.replace(0, np.nan, inplace=True)

In [139]:
final_df = final_df.merge(metadata, how='outer')

In [140]:
final_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1135 entries, 0 to 1134
Data columns (total 10 columns):
id                       1135 non-null object
full_date                1036 non-null object
patient__autocolor       792 non-null float64
province__autocolor      873 non-null object
SEX__autocolor           1032 non-null object
AGE__autocolor           1033 non-null object
day                      1036 non-null Int64
month                    1036 non-null Int64
year                     1036 non-null Int64
DEPARTMENT__autocolor    666 non-null object
dtypes: Int64(3), float64(1), object(6)
memory usage: 100.9+ KB


In [141]:
final_df.tail()

,id,full_date,patient__autocolor,province__autocolor,SEX__autocolor,AGE__autocolor,day,month,year,DEPARTMENT__autocolor
1130,20444230,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,TRABAJADORES_OCT_NOV
1131,20284978,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,INGRESADO
1132,20435764,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2300
1133,20458365,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,C900
1134,20097906,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,REINF_RUN12


In [142]:
def extract_lineage(sample):
    pango_file = os.path.join('/home/laura/ANALYSIS/VARIANT_CALLING/COVID/Annotation/pangolin', str(sample) + ".lineage.csv")
    if os.path.exists(pango_file):
        df = pd.read_csv(pango_file)
        return df.lineage[0]
    else:
        return None

In [143]:
final_df['Lineage__autocolor'] = final_df.apply(lambda x: extract_lineage(x['id']), axis=1)

In [144]:
def extract_EU(sample):
    eu_file = os.path.join('/home/laura/ANALYSIS/VARIANT_CALLING/COVID/Annotation/user', str(sample) + ".tsv")
    if os.path.exists(eu_file):
        df = pd.read_csv(eu_file, sep="\t")
        if 'Nextrain_all' in df.columns:
            eulist = df.Nextrain_all.dropna().tolist()
        return (',').join([(x + "-" + str(eulist.count(x))) for x in set(eulist)])
    else:
        return None

In [145]:
final_df['Nextrain__autocolor'] = final_df.apply(lambda x: extract_EU(x['id']), axis=1)

In [146]:
def extract_resistance(sample):
    res_file = os.path.join('/home/laura/ANALYSIS/VARIANT_CALLING/COVID/Annotation/user_aa', str(sample) + ".tsv")
    if os.path.exists(res_file):
        df = pd.read_csv(res_file, sep="\t")
        if 'resistance_covid' in df.columns:
            relist = df.resistance_covid.dropna().tolist()
        return (',').join([(x + "-" + str(relist.count(x))) for x in set(relist)])
    else:
        return None

In [147]:
final_df['Resistance__autocolor'] = final_df.apply(lambda x: extract_resistance(x['id']), axis=1)

In [148]:
def extract_faulty(sample):
    sample_report = os.path.join(compare_folder, "report_samples.tsv")
    df = pd.read_csv(sample_report, sep="\t")
    df['sample'] = df['sample'].astype(str)
    if str(sample) in df['sample'].values:
        return round(df[df['sample'] == str(sample)]['faulty_frac'].values[0], 3)
    else:
        return None

In [149]:
final_df['FaultyFraction__autocolor'] = final_df.apply(lambda x: extract_faulty(x['id']), axis=1)

In [150]:
#query = ['20097501','20097906','20099990','20101024','20107580','20143658','20214237','20247923','20398264','20425128','20425595','20428122','20431031','20435733','20450595','20450832','20455085','20455763','20455870','20458052','20458113','20458114','20458118','20458269','20458270','20458365','20458528','20458547','20458560','20458569','20458576','20458635','20458638','20458668','20458686','20458702','20461287','20461287-BIS','20461290','20461293','20461349','20461351','20461375','20461389','20461389-BIS','20461392','20461416','20461563','20461601','20461621','20461637','20461637-BIS','20461689','20461747','20461783','20461785','20461790','20461801','20461861','20461869','20461870','20461871','20461886','20461904','20461905','20461907','20461908','20461926','20461928','20461937','20461953','20461968','20464095','20464109','20464109-BIS','20464197','20464303','20464456','20464490','20464493','20464516','20464517','20464519','20464521','20464532','20464534','20464567','20464570','20464580','20464595','20464599','20464600','20464612','20464613','20464614','20464636']
#query_dario = [str(x) for x in query_dario]


In [151]:
#final_df['run12Maldito__autocolor'] = final_df.apply(lambda x: x['id'] in query, axis=1)


In [152]:
before_june = final_df[final_df.month < 7]['id'].tolist()
before_june = [str(x) for x in before_june]
final_df['before_june_2020__autocolor'] = final_df.apply(lambda x: x['id'] in before_june, axis=1)

In [153]:
final_df = final_df.drop_duplicates(subset=['id'], keep="first")
out_file = os.path.join(compare_folder, 'wgs_microreact.csv')
final_df.to_csv(out_file, index=False)

In [154]:
final_df.head()

,id,full_date,patient__autocolor,province__autocolor,SEX__autocolor,AGE__autocolor,day,month,year,DEPARTMENT__autocolor,Lineage__autocolor,Nextrain__autocolor,Resistance__autocolor,FaultyFraction__autocolor,before_june_2020__autocolor
0,20060161,09/03/2020,174.0,Madrid,H,84,9,3,2020,GRAVES,A,,,0.004,True
1,20060178,09/03/2020,175.0,Madrid,H,76,9,3,2020,GRAVES,A,,,0.004,True
2,20062236,10/03/2020,176.0,Madrid,M,96,10,3,2020,GRAVES,B.2,,,0.001,True
3,20062563,10/03/2020,177.0,Madrid,H,78,10,3,2020,GRAVES,B.1.5,20A-2,,0.002,True
4,20063795,12/03/2020,178.0,Madrid,H,85,12,3,2020,GRAVES,A.5,19B-2,,0.001,True
